In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import os
import vtk
from skimage import io, transform, measure, filters
import concurrent.futures

C:\Users\Moji\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
class ImageProcessor:
    """Handles the image processing operations."""

    @staticmethod
    def process_folder(folder_path, scale_factor, z_scale_factor):
        """Processes a folder of images, applies scaling, and performs marching cubes.
        
        Args:
            folder_path (str): The path to the folder containing images.
            scale_factor (float): The scale factor for XY axes.
            z_scale_factor (float): The scale factor for the Z-axis.

        Returns:
            str: The path to the saved .npz file with processed data.
        """
        npz_dir = "NPZ_files"
        os.makedirs(npz_dir, exist_ok=True)

        file_paths = ImageProcessor._get_image_files(folder_path)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            slices = list(executor.map(lambda f: ImageProcessor._load_and_rescale_image(f, scale_factor), file_paths))

        slices = np.array(slices)
        slices = filters.gaussian(slices, sigma=0.5)

        spacing = (z_scale_factor, 1.0, 1.0)
        vertices, faces, _, _ = measure.marching_cubes(slices, spacing=spacing)

        folder_name = os.path.basename(folder_path)
        npz_file_path = os.path.join(npz_dir, f"{folder_name}.npz")
        np.savez(npz_file_path, vertices=vertices, faces=faces)

        return npz_file_path
    @staticmethod
    def _load_and_rescale_image(file_path, scale_factor):
        """Loads and rescales an image.
        
        Args:
            file_path (str): The path to the image file.
            scale_factor (float): The scale factor to apply.

        Returns:
            np.ndarray: The rescaled image.
        """
        img = io.imread(file_path, as_gray=True)
        return transform.rescale(img, scale_factor, anti_aliasing=True)

    @staticmethod
    def _get_image_files(folder_path):
        """Gets PNG image file paths from the folder.
        
        Args:
            folder_path (str): The folder containing images.

        Returns:
            list: List of image file paths.
        """
        file_paths = [os.path.join(folder_path, filename)
                      for filename in sorted(os.listdir(folder_path))
                      if filename.endswith(".png")]

        if not file_paths:
            raise ValueError("No PNG images found in the selected folder.")
        return file_paths

In [3]:
class Visualizer:
    """Handles 3D visualization using VTK."""

    @staticmethod
    def visualize_3d(npz_file_path):
        """Visualizes the 3D data from an npz file.
        
        Args:
            npz_file_path (str): The path to the .npz file containing 3D data.
        """
        try:
            data = np.load(npz_file_path)
            vertices = data['vertices']
            faces = data['faces']

            points = vtk.vtkPoints()
            for vertex in vertices:
                points.InsertNextPoint(vertex)

            polydata = vtk.vtkPolyData()
            polydata.SetPoints(points)

            faces_array = vtk.vtkCellArray()
            for face in faces:
                triangle = vtk.vtkTriangle()
                for i in range(3):
                    triangle.GetPointIds().SetId(i, face[i])
                faces_array.InsertNextCell(triangle)

            polydata.SetPolys(faces_array)

            mapper = vtk.vtkPolyDataMapper()
            mapper.SetInputData(polydata)
            mapper.ScalarVisibilityOff()

            actor = vtk.vtkActor()
            actor.SetMapper(mapper)

            renderer = vtk.vtkRenderer()
            renderer.AddActor(actor)

            render_window = vtk.vtkRenderWindow()
            render_window.AddRenderer(renderer)

            interactor = vtk.vtkRenderWindowInteractor()
            interactor.SetRenderWindow(render_window)

            renderer.ResetCamera()
            render_window.Render()
            interactor.Start()
        except Exception as e:
            raise RuntimeError(f"Error during visualization: {e}")

In [4]:
class ImageProcessingApp:
    """Main application for image processing with a Tkinter GUI."""

    def __init__(self, master):
        self.master = master
        master.title("3D Image Processor")

        self._init_ui()

        self.folder_path = None
        self.npz_file_path = None

    def _init_ui(self):
        """Initializes the UI components."""
        self.scale_label = tk.Label(self.master, text="Scale Factor (XY):")
        self.scale_label.pack()
        self.scale_slider = tk.Scale(self.master, from_=0.1, to=1.0, resolution=0.01, orient="horizontal")
        self.scale_slider.set(0.25)
        self.scale_slider.pack()

        self.z_scale_label = tk.Label(self.master, text="Z-Scale Factor:")
        self.z_scale_label.pack()
        self.z_scale_slider = tk.Scale(self.master, from_=0.5, to=5.0, resolution=0.1, orient="horizontal")
        self.z_scale_slider.set(1.0)
        self.z_scale_slider.pack()

        self.folder_button = tk.Button(self.master, text="Select Folder", command=self.select_folder)
        self.folder_button.pack()

        self.process_button = tk.Button(self.master, text="Process Images", command=self.process_images)
        self.process_button.pack()

    def select_folder(self):
        """Handles folder selection."""
        self.folder_path = filedialog.askdirectory()
        if self.folder_path:
            messagebox.showinfo("Folder Selected", f"Selected folder: {self.folder_path}")

    def process_images(self):
        """Processes images and visualizes them in 3D."""
        if not self.folder_path:
            messagebox.showwarning("No Folder", "Please select a folder first.")
            return

        scale_factor = self.scale_slider.get()
        z_scale_factor = self.z_scale_slider.get()

        try:
            self.npz_file_path = ImageProcessor.process_folder(self.folder_path, scale_factor, z_scale_factor)
            messagebox.showinfo("Processing Complete", f"Saved processed data to: {self.npz_file_path}")
            Visualizer.visualize_3d(self.npz_file_path)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

In [5]:
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingApp(root)
    root.mainloop()